In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from vivarium_research_prl.noise import corruption, fake_names
from vivarium_research_prl.find_kids import datasets

!date
!whoami
!uname -a
!pwd

Wed 07 Dec 2022 10:18:43 PM PST
ndbs
Linux int-slurm-sarchive-p0009 5.4.0-88-generic #99-Ubuntu SMP Thu Sep 23 17:29:00 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/linkage/wic_case_study


In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
from vivarium_research_prl import noise

In [34]:
import datetime as dt

# Load census data and state table for year 2020

In [3]:
project_output_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = 'results/special_last_names/florida/2022_10_14_10_49_32/population_table/'
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -l $output_dir

total 32224
-rw-rw-r-- 1 albrja   IHME-Simulationscience 12622072 Oct 20 23:08 decennial_census.hdf
-rwxrwxrwx 1 beatrixh IHME-Simulationscience 20364830 Nov 14 16:42 state_table.hdf


In [4]:
decennial_census_path = f'{output_dir}/decennial_census.hdf'
state_table_path = f'{output_dir}/state_table.hdf'

df_census_orig = pd.read_hdf(decennial_census_path, 'year_2020')
df_state_table = pd.read_hdf(state_table_path, 'ymd_2020_4_1')
print(f'{df_census_orig.shape=}')
print(f'{df_state_table.shape=}')

df_census_orig.shape=(47444, 10)
df_state_table.shape=(50000, 27)


# Get a random seed and Generator

In [5]:
sq = np.random.SeedSequence()
sq.entropy

263281533040163012496346764338284145225

In [6]:
sq = np.random.SeedSequence(66624024798819663709061712465147975287)
print(sq.entropy)
rng = np.random.default_rng(sq)
rng

66624024798819663709061712465147975287


Generator(PCG64) at 0x7F5DF76FAE40

# Generate WIC data and decennial census data

In [7]:
df_census = datasets.get_census_data_with_missing_kids(df_census_orig, 0.05, rng)
df_wic = datasets.generate_wic_data(df_state_table, rng)
print(f"{df_census.shape=}")
print(f"{df_wic.shape=}")

df_census.shape=(47326, 10)
df_wic.shape=(617, 10)


In [13]:
df_census

,first_name,last_name,age,date_of_birth,address,zipcode,relation_to_household_head,sex,race_ethnicity,middle_initial
0,Margaret,Clark,68.0,1951-07-27,"1344 winoka rd brooksville, fl",34601,Reference person,Female,Black,J
1,Jeffrey,Littlejohn,52.0,1967-05-03,"927 23rd st clearwater, fl",34698,Reference person,Male,Black,V
2,Briana,Jackson,13.0,2006-09-07,"927 23rd st clearwater, fl",34698,Biological child,Female,Black,A
3,Benjamin,Cox,21.0,1998-10-21,"927 23rd st clearwater, fl",34698,Stepchild,Male,Black,D
4,Willie,Tucker,72.0,1947-10-09,"8904 167th place fleming island, fl",32003,Reference person,Male,White,J
...,...,...,...,...,...,...,...,...,...,...
49993,Thomas,Gutierrez,29.0,1990-11-16,"2210 henn hyde rd ne hollywood, fl",33021,Institutionalized GQ pop,Male,Latino,B
49994,Marcus,Roman,31.0,1988-07-08,"2210 henn hyde rd ne hollywood, fl",33021,Institutionalized GQ pop,Male,Multiracial or Other,S
49997,Christian,Rosales,36.0,1983-12-16,"701 haber rd vero beach, fl",32968,Institutionalized GQ pop,Male,Latino,C
49998,Phillip,Morton,34.0,1985-06-11,"114 s frnt st fort myers, fl",33919,Institutionalized GQ pop,Male,White,J


In [47]:
df_census.dtypes

first_name                            object
last_name                             object
age                                  float64
date_of_birth                 datetime64[ns]
address                               object
zipcode                               object
relation_to_household_head            object
sex                                   object
race_ethnicity                        object
middle_initial                        object
dtype: object

In [9]:
df_wic

,first_name,middle_name,last_name,date_of_birth,sex,race_ethnicity,address,zipcode,household_id,wic_id
82,Sadie,Katia,Tidwell,2017-10-15,Female,Black,"w 4th st north port, fl",34287,48,1
83,Liliana,Addisyn,Marshall,2019-12-03,Female,Black,"w 4th st north port, fl",34287,48,2
174,Holly,Emma,Yount,2019-05-17,Female,White,"7944 se 62nd ave unincorporated, fl",32824,88,3
183,Jose,Zamir,Ulrich,2019-12-12,Male,White,"5124 braemar st spring hill, fl",34606,91,4
364,Angel,Jaquan,Gutierrez,2015-05-12,Male,Latino,"2505 easy st tampa, fl",33647,173,5
...,...,...,...,...,...,...,...,...,...,...
48231,Olivia,Alice,Deroest,2019-06-01,Female,White,"northeast tillamook street fort myers, fl",33966,20367,613
48261,Emma,Celine,Bland,2018-08-22,Female,Black,"719 orangevale ct unincorporated, fl",33415,20379,614
48442,Frederick,Cameron,Rodriguez,2019-06-04,Male,Latino,"5765 heards forest dr crestview, fl",32539,20452,615
48457,Emma,Maya,Bergman,2017-08-31,Female,White,"107 brown ave st. petersburg, fl",33704,20458,616


In [48]:
df_wic.dtypes

first_name                object
middle_name               object
last_name                 object
date_of_birth     datetime64[ns]
sex                       object
race_ethnicity            object
address                   object
zipcode                   object
household_id               int64
wic_id                     int64
dtype: object

# See how many kids are in census data

Only 2236. If I restrict to these, it would be a very small linking task. However, if ages and birthdates are noisy, then we'd have to search the rest of the records after matching these.

    
## Questions about linking

1. If ages/birthdays are noisy, what's a good strategy for finding missing kids in a "cascade" approach?

In [50]:
(df_census.age < 5).sum()

2236

# Add noise to data

## Noise for census in priority order:

1. First name (keyboard, phonetic, fake)
1. Last name (keyboard, phonitic, fake)
1. Date of birth (switch month/day, wrong number or person)
1. Zipcode
1. Address
1. Sex
1. Age
1. Race/Ethnicity
1. Middle initial

## Noise for WIC data in priority order:

1. First name (keyboard, phonetic, fake, OCR)
1. Last name (keyboard, phonitic, fake, OCR)
1. Date of birth (switch month/day, wrong number or person, OCR)
1. Zipcode
1. Address
1. Sex
1. Race/Ethnicity
1. Middle name
1. Household ID and WIC ID??

## Questions about adding noise:

1. What are good default values for the token corruption probabilities?

1. How should I handle combining the row corruption probability with the token corruption probabilities? To get a desired fraction of rows corrupted, you need to modify a larger fraction of rows, e.g. divide the row fraction by the token corruption probability or something. (The question is not about how to do the math, but about where to do the math, or if we need to do the math.)

1. How should I handle the datetime64 datatype in date of birth?

    1. Does Splink (and other packages) handle this datatype directly, or do they need to be converted to strings?
    1. How do I switch the month and day in a fraction of rows?
    1. If a swapped month and day lead to an invalid date in the format YYYY-MM-DD, do we keep the invalid date, or should this get fixed automatically since it's obviously invalid? Would correcting obvious errors like this be part of the preprocessing pipeline? E.g., just like there's name and address standardization, there would be date standardization.
    1. Maybe the problem is that the data is already "too clean" because I'm just starting with the ground truth, which is stored as datetime64 objects in a standard format (e.g., so I can't switch the month and day to invalid values without getting an error or changing the format of every row). Perhaps part of the initial "data creation" process should be to convert these to strings, which have less structure and may be easier to manipulate, and then one of the first preprocessing steps would be to standardize the dates, eliminiating any impossible values.

1. What **other** ways could birthdays be corrupted besides swapping month and day?

1. There are **lots** of possible parameters to specify for noise functions. Where is the best place to specify these, how do I pass them around to different parts of the code, and how do I make a user-friendly (for now, me-friendly) interface to handle them?

1. How should I add noise to zipcodes?

1. How should I add noise to addresses?

    1. Also, should I try to do any address standardization? Are there available packages that already do this?
    1. Can I pull addresses from the 2030 census file to 

In [31]:
df_census.date_of_birth

0       1951-07-27
1       1967-05-03
2       2006-09-07
3       1998-10-21
4       1947-10-09
           ...    
49993   1990-11-16
49994   1988-07-08
49997   1983-12-16
49998   1985-06-11
49999   1928-08-14
Name: date_of_birth, Length: 47326, dtype: datetime64[ns]

In [44]:
df_census.date_of_birth.astype(str)[0]

'1951-07-27'

In [46]:
df_census.date_of_birth.dt.strftime('%Y-%d-%m')

0        1951-27-07
1        1967-03-05
2        2006-07-09
3        1998-21-10
4        1947-09-10
            ...    
49993    1990-16-11
49994    1988-08-07
49997    1983-16-12
49998    1985-11-06
49999    1928-14-08
Name: date_of_birth, Length: 47326, dtype: object

In [40]:
col = df_census.date_of_birth.copy()
apply_noise_to_column(
        col, 0.02, rng, dt.datetime.strftime, ('%Y-%d-%m',))
col

0        1951-07-27 00:00:00
1        1967-05-03 00:00:00
2        2006-09-07 00:00:00
3        1998-10-21 00:00:00
4        1947-10-09 00:00:00
                ...         
49993    1990-11-16 00:00:00
49994    1988-07-08 00:00:00
49997    1983-12-16 00:00:00
49998    1985-06-11 00:00:00
49999    1928-08-14 00:00:00
Name: date_of_birth, Length: 47326, dtype: object

In [42]:
pd.to_datetime(col)

ParserError: month must be in 1..12: 1941-18-09 present at position 38

In [41]:
df_census.date_of_birth.compare(col)

,self,other
38,1941-09-18,1941-18-09
133,1994-08-20,1994-20-08
173,2013-01-15,2013-15-01
212,1960-03-04,1960-04-03
217,1974-06-22,1974-22-06
...,...,...
49839,1925-02-09,1925-09-02
49862,1946-07-25,1946-25-07
49922,2003-04-08,2003-08-04
49930,1924-05-15,1924-15-05


In [ ]:
# df['date'] = df['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%d-%m %H:%M:%S'))

In [39]:
def apply_noise_to_column(
    column, row_prob, rng, noise_function, args=(), kwargs={}
):
    """Apply a noise function that operates on scalars to a fraction of
    rows in a single column. Modifies column in place.
    """
    corrupted = rng.random(len(column)) < row_prob
    column.loc[corrupted] = column.loc[corrupted].map(
        lambda x: noise_function(x, *args, **kwargs))
        
def add_noise_to_census(df_census, random_state):
    rng = np.random.default_rng(random_state)
    df_census = df_census.copy()
    
    # First name
    apply_noise_to_column(
        df_census['first_name'], 0.01, rng, corruption.keyboard_corrupt, (0.05, 0.02))
    apply_noise_to_column(
        df_census['first_name'], 0.001, rng, corruption.phonetic_corrupt, (0.05,))
    
    # Last name
    apply_noise_to_column(
        df_census['last_name'], 0.01, rng, corruption.keyboard_corrupt, (0.05, 0.02))
    apply_noise_to_column(
        df_census['last_name'], 0.001, rng, corruption.phonetic_corrupt, (0.05,))
    
    # Date of birth
    apply_noise_to_column(
        df_census['date_of_birth'], 0.02, rng, dt.datetime.strftime, ('%Y-%d-%m',))

In [28]:
noise_settings = {
    'corruption.keyboard_corrupt': [0.01, (0.05, 0.02)],
    'corruption.phonetic_corrupt': [0.001, (0.05,)],
}

def add_noise_to_column(column, noise_settings, random_state=None):
    if random_state is None:
        rng = np.random.default_rng(random_state)
    new_column = column.copy()
    for funckey, (row_prob, args) in noise_settings.items():
        module_name, funcname = funckey.split('.')
        func = getattr(getattr(noise, module_name), funcname)
        corrupted = rng.random(len(column)) < row_prob
        new_column.loc[corrupted] = new_column.loc[corrupted].map(lambda x: func(x, *args))
    return new_column

noised_names = add_noise_to_column(df_census.first_name, noise_settings)
noised_names

0         Margaret
1          Jeffrey
2           Briana
3         Benjamin
4           Willie
           ...    
49993       Thomas
49994       Marcus
49997    Christian
49998      Phillip
49999    Catherine
Name: first_name, Length: 47326, dtype: object

In [29]:
df_census.first_name.compare(noised_names)

,self,other
14,Tiffany,Tjffany
190,Jacqueline,Jacquepine
511,Lee,Le
820,Clara,Clada
1862,Stephanie,Shephanie
...,...,...
47330,Roberta,Roferta
47411,Kelly,Keply
48966,Eric,Erix
49291,Timothy,Timodh


In [17]:
dir(noise)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'corruption',
 'fake_names']